# Data cleaning and merging dataframes

## Loading multiple datasets

### Classical way

Reading file by file

In [19]:
import pandas as pd

path = '../eniac-data/'
orderlines = pd.read_csv(path + 'orderlines.csv')
orders = pd.read_csv(path + 'orders.csv')
brands = pd.read_csv(path + 'brands.csv')
products = pd.read_csv(path + 'products.csv')

In [505]:
import pandas as pd

orders = pd.read_csv("orders.csv")
orderlines = pd.read_csv("orderlines.csv")
products = pd.read_csv("products.csv")


### Another way

Using `os` and a loop to read all the files from a directory. It's also possible to read only files with a certain extension (like `.csv`):

In [21]:
import pandas as pd
import os
path = '../eniac-data/'
path, dirs, files = next(os.walk(path))
print(files)

# remove non-csv files
for file in files:
    if not file.endswith("csv"):
        files.remove(file)
        print(file)

['products.csv', 'orders.csv', 'orderlines.csv', 'Eniac_DataFrames_exploration.ipynb', '3-empty.ipynb', 'brands.csv']
Eniac_DataFrames_exploration.ipynb


In [22]:
# create empty list 
df_list = []

# append datasets to the list
for file in files:
    temp_df = pd.read_csv(path + file, sep=',')
    df_list.append(temp_df)

# show results
products, orders, orderlines, brands = df_list[0], df_list[1], df_list[2], df_list[3]

ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2


## Data quality

### Missing values

In [ ]:
def check_missing_values(data): 
    print('Missing values:' + '\n' + str(data.isna().sum()))

In [ ]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    print('\n')

### Duplicates

In [ ]:
def check_duplicates(data): 
    print('Duplicated rows: ', data.duplicated().sum())

In [ ]:
for i in range(len(df_list)): 
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    print('\n')

Based on our initial exploration we need to find a way to deal with our missing values. 

Do we have other problems like data inconsistency? 

* Example: problems with the `price` column in the `orderlines` dataframe.

### Data inconsistency

In [ ]:
for i in range(len(df_list)): 
    
    print(f'File name: {files[i]}')
    check_missing_values(df_list[i])
    check_duplicates(df_list[i])
    df_list[i].info()
    print('\n')

Some aspects that do not make a lot of sense:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be floats. 
    * Quite a lot of missing values: 
        * `description`: 7 missing values. Maybe that could be inferred from the product name?
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 

    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` mathces with the created dates for `orderlines`. 
    
    
* **orderlines**: 
    * `unit_price` has to be a float, something wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it with matches with the `orders` dataset. 
    
    
* **brands**: looks fine. 


Where do we have to start? Since `orders` and `orderlines` seem very crucial to the analysis, we will start by checking that they match.

## Cleaning orders

In [26]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])
print(orders)

        order_id        created_date  total_paid            state  \
0         241319 2017-01-02 13:35:40       44.99        Cancelled   
1         241423 2017-11-06 13:10:02      136.15        Completed   
2         242832 2017-12-31 17:40:03       15.76        Completed   
3         243330 2017-02-16 10:59:38       84.98        Completed   
4         243784 2017-11-24 13:35:19      157.86        Cancelled   
...          ...                 ...         ...              ...   
226904    527397 2018-03-14 13:56:38       42.99      Place Order   
226905    527398 2018-03-14 13:57:25       42.99  Shopping Basket   
226906    527399 2018-03-14 13:57:34      141.58  Shopping Basket   
226907    527400 2018-03-14 13:57:41       19.98  Shopping Basket   
226908    527401 2018-03-14 13:58:36       18.98      Place Order   

        check_orders  
0               True  
1               True  
2               True  
3               True  
4               True  
...              ...  
226904    

Some data I could extract from here to do a data quality check with `orderlines`: 

* the sum of `total_paid` on both datasets should be the same
* the initial and last date should be the same

In [27]:
min(orders['created_date'])

Timestamp('2017-01-01 00:07:19')

In [28]:
max(orders['created_date'])

Timestamp('2018-03-14 13:58:36')

In [29]:
sum(orders['total_paid'])

nan

I forgot to deal with missing values on our orders dataset!

In [30]:
orders.isna().sum()

order_id        0
created_date    0
total_paid      5
state           0
check_orders    0
dtype: int64

In [31]:
# let's filter them out
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state,check_orders
127701,427314,2017-11-20 18:54:39,NaN,Pending,True
132013,431655,2017-11-22 12:15:24,NaN,Pending,True
147316,447411,2017-11-27 10:32:37,NaN,Pending,True
148833,448966,2017-11-27 18:54:15,NaN,Pending,True
149434,449596,2017-11-27 21:52:08,NaN,Pending,True


These orders have a state of "Pending", so I can exclude them from our dataset. We are interested on the completed ones.

In [32]:
orders.dropna(inplace=True)

Now my dataset is clean. And the total paid is: 

In [33]:
orders['total_paid'].sum()

129130877.18

## Cleaning orderlines

First let's transform our date time. 

In [34]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [35]:
min(orderlines['date'])
# orders: Timestamp('2017-01-01 00:07:19')

Timestamp('2017-01-01 00:07:19')

In [36]:
max(orderlines['date'])
# orders: Timestamp('2018-03-14 13:58:36')

Timestamp('2018-03-14 13:58:36')

Yes, it seems it match. Now let's test our total_paid. 

To get this value, we will have to calculate a new column, total price for each row. It would be `product_quantity` * `unit_price`. I can not multiply a two different types of data. So I will have to transform the unit price to a numeric one. 

In [37]:
orderlines['product_quantity'] * pd.to_numeric(orderlines['unit_price'])

ValueError: Unable to parse string "1.137.99" at position 6

While trying to transform this column to numeric an error appears. From the error message, "Unable to parse string "1.137.99" at position 6", it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and python only admits one dot per number: the _decimal_ separator!

There are a lot of different ways to solve this problem. The first thing I will do is to count how many dots appear for each number using string operations. If price has more than one dot, I will consider it corrupted. 

In [38]:
# we create a copy of the dataset
ol_temp = orderlines.copy()

# create a new column with the amount of dots in the unit_price column
ol_temp['dots'] = orderlines['unit_price'].str.count('\.')

# show the rows with more than one dot
ol_temp.query('dots > 1')

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21,2
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42,2
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47,2
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00,2
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11,2
...,...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33,2
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15,2
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56,2
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51,2


Our theory about the thousands separators is confirmed. How can we solve this problem? 

My approach would be to remove all the dots for all the unit_prices, and then add a dot before the last 2 digits to all the rows. Then I will transform it into numeric values.

In [39]:
orderlines = orderlines.assign(unit_price_nd = orderlines['unit_price'].str.replace('\.','', regex=True))
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374


In [40]:
orderlines['digits'] = orderlines['unit_price_nd'].str[-2:]
orderlines['integers'] = orderlines['unit_price_nd'].str[:-2]
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,99,18
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,00,399
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,05,474
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,39,68
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,74,23


In [41]:
orderlines['new_unit_price'] = orderlines['integers'] + '.' + orderlines['digits']
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,digits,integers,new_unit_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,99,18,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,00,399,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,05,474,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,39,68,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,74,23,23.74


In [42]:
# last step is to transform it to numeric
orderlines['unit_price'] = pd.to_numeric(orderlines['new_unit_price'])
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
 7   unit_price_nd     293983 non-null  object        
 8   digits            293983 non-null  object        
 9   integers          293983 non-null  object        
 10  new_unit_price    293983 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 24.7+ MB


Now it is time to see if it is the same than the orders files. 

In [43]:
orderlines.drop(['unit_price_nd','digits','integers','new_unit_price'], axis=1, inplace=True)

orderlines['total_price'] = orderlines['product_quantity'] * orderlines['unit_price']
sum(orderlines['total_price'])
# 129159615.07000001

128776222.0298095

What!? After all this work the values do not match! How can we figure it out where is the problem? 

## Matching `orders` and `orderlines`

It is possible that some orders exist in one dataset but not in the other one. This would be a potential source for this price mismatch. Let's solve it!

We first create a new column in the `orderlines` dataset using `assign`. We also use `isin()` to create a boolen value (True/False) that checks whether the `id_order` is present in the `orders` dataset:

In [80]:
(orderlines.
 assign(check_orders = orderlines.id_order.isin(orders.order_id))
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99,True
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00,True
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05,True
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39,True
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74,True
...,...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,42.99,True
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,141.58,True
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,19.98,True
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,19.99,True


Then, using `.query` we select rows where the value in this new column is `False`:

In [83]:
orderlines.assign(check_orders = orderlines.id_order.isin(orders.order_id)).query("check_orders==False")

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders


It looks like 240 rows in `orderlines` come from orders not present in the `orders` dataset. This is quite inconsistent, since the `orders` dataset should be the one and only source of truth for orders: if an order is not there, it should not exist. We will try to find out why that happened, but for now, let's just remove those "ghost" orders:

In [75]:
orderlines = (orderlines.
              assign(check_orders = orderlines.id_order.isin(orders.order_id)).
              query("check_orders==True")
             )
print(orderlines)

             id  id_order  product_id  product_quantity      sku  unit_price  \
0       1119109    299539           0                 1  OTT0133       18.99   
1       1119110    299540           0                 1  LGE0043      399.00   
2       1119111    299541           0                 1  PAR0071      474.05   
3       1119112    299542           0                 1  WDT0315       68.39   
4       1119113    299543           0                 1  JBL0104       23.74   
...         ...       ...         ...               ...      ...         ...   
293978  1650199    527398           0                 1  JBL0122       42.99   
293979  1650200    527399           0                 1  PAC0653      141.58   
293980  1650201    527400           0                 2  APP0698        9.99   
293981  1650202    527388           0                 1  BEZ0204       19.99   
293982  1650203    527401           0                 1  APP0927       13.99   

                      date  total_price

Now let's look at this problem in the opposite direction: are there orders in the `orders` dataset not prsent in `orderlines`?

In [85]:
(orders.
 assign(check_orders = orders.order_id.isin(orderlines.id_order)).
 query("check_orders==False")
)

,order_id,created_date,total_paid,state,check_orders


There are more than 22000 orders in the `orders` dataset that are not present on the `orderlines` dataset!!! We can try to find out why by looking at the state of these orders:

In [58]:
(orders.
 assign(check_orders = orders.order_id.isin(orderlines.id_order)).
 query("check_orders==False").
 state.value_counts()
)

Series([], Name: state, dtype: int64)

It looks like most of them are orders that were not fully completed: the products were left in the shopping basket or the order was "placed" but maybe not paid (hence the state "Place Order". Some of them were "Completed", though. 

This will require further research, and we might have to come back to these orders if we have to explore consumer behaviour (e.g. why are orders left in the shopping basket?), but for now, for the sake of data quality, let's drop all of these unmatched orders:

In [57]:
orders = (orders.
          assign(check_orders = orders.order_id.isin(orderlines.id_order)).
          query("check_orders==True")
         )
print(orders)

        order_id        created_date  total_paid            state  \
0         241319 2017-01-02 13:35:40       44.99        Cancelled   
1         241423 2017-11-06 13:10:02      136.15        Completed   
2         242832 2017-12-31 17:40:03       15.76        Completed   
3         243330 2017-02-16 10:59:38       84.98        Completed   
4         243784 2017-11-24 13:35:19      157.86        Cancelled   
...          ...                 ...         ...              ...   
226904    527397 2018-03-14 13:56:38       42.99      Place Order   
226905    527398 2018-03-14 13:57:25       42.99  Shopping Basket   
226906    527399 2018-03-14 13:57:34      141.58  Shopping Basket   
226907    527400 2018-03-14 13:57:41       19.98  Shopping Basket   
226908    527401 2018-03-14 13:58:36       18.98      Place Order   

        check_orders  
0               True  
1               True  
2               True  
3               True  
4               True  
...              ...  
226904    

Let's now check again if the total paid matches:

In [51]:
orders.total_paid.sum()

129130877.18

In [34]:
orderlines.total_price.sum()

128659393.77

STILL NOT MATCHING!!! This is outrageous. Let's keep exploring.

## Solving the price mismatch

Let's merge both datasets and compare, order by order, the `total_price`. We will call this new merged dataset `orders_info`.

*Note: To merge `orderlines` with `orders`, we have to group and aggregate `orderlines`!

In [55]:
orders_info = (
orderlines
    .groupby('id_order')
    .agg({'total_price':'sum'})
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    
)
orders_info

,total_price,order_id,created_date,total_paid,state,check_orders
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True
...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True


Now that the `total` from both datasets is in the same dataframe, we can create a new column with the difference:

In [89]:
orders_info['price_difference'] = orders_info['total_paid'] - orders_info['total_price']
orders_info

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True,0.00
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True,6.99
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True,4.99
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True,6.99
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True,3.99
...,...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True,0.00
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True,0.00
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True,0.00
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True,0.00


In [90]:
orders_info.price_difference.describe()

count    204691.000000
mean          2.303391
std         297.461827
min      -36159.900000
25%           0.000000
50%           0.000000
75%           0.010000
max       90898.610000
Name: price_difference, dtype: float64

Looks like the maximum and minimun price differences are huge: some orders are really corrupted. But we also see from the mean and the quartiles that the price difference is neglegible for most orders.

## Challenge: Remove outliers

Decide on a criteria for removing orders whenever you cannot trust the price difference between `orders` and `orderlines`. 

In [96]:
orders_info.columns

Index(['total_price', 'order_id', 'created_date', 'total_paid', 'state',
       'check_orders', 'price_difference'],
      dtype='object')

In [604]:
orders_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204691 entries, 0 to 204690
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   total_price       204691 non-null  float64       
 1   order_id          204691 non-null  int64         
 2   created_date      204691 non-null  datetime64[ns]
 3   total_paid        204691 non-null  float64       
 4   state             204691 non-null  object        
 5   check_orders      204691 non-null  bool          
 6   price_difference  204691 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 19.2+ MB


In [668]:
orders_info =orders_info_copy.query('0<=price_difference<150')
orders_info.drop(['float_col'], axis=1)


,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True,0.00
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True,6.99
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True,4.99
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True,6.99
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True,3.99
...,...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True,0.00
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True,0.00
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True,0.00
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True,0.00


In [622]:
print(orders_info)

        total_price  order_id        created_date  total_paid  \
0             44.99    241319 2017-01-02 13:35:40       44.99   
1            129.16    241423 2017-11-06 13:10:02      136.15   
2             10.77    242832 2017-12-31 17:40:03       15.76   
3             77.99    243330 2017-02-16 10:59:38       84.98   
4            153.87    243784 2017-11-24 13:35:19      157.86   
...             ...       ...                 ...         ...   
204686        42.99    527397 2018-03-14 13:56:38       42.99   
204687        42.99    527398 2018-03-14 13:57:25       42.99   
204688       141.58    527399 2018-03-14 13:57:34      141.58   
204689        19.98    527400 2018-03-14 13:57:41       19.98   
204690        13.99    527401 2018-03-14 13:58:36       18.98   

                  state  check_orders  price_difference  float_col  
0             Cancelled          True              0.00         44  
1             Completed          True              6.99        136  
2           

In [614]:
orders_info.price_difference.describe()

count    199927.000000
mean          1.447792
std           3.072423
min           0.000000
25%           0.000000
50%           0.000000
75%           0.010000
max         132.210000
Name: price_difference, dtype: float64

Save the data once you are sure you can trust it!

In [39]:
orderlines.to_csv(path + 'clean/orderlines_cl.csv', index=False)
orders.to_csv(path + 'clean/orders_cl.csv', index=False)

## Challenge: Cleaning products

Now it is time to clean the products dataset. Let's do a quick review of the things where wrong: 

In [43]:
print(products.info())
check_duplicates(products)
check_missing_values(products)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
None
Duplicated rows:  8746
Missing values:
sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64


Looking at this overview, we can see that there are different things that has to be changed: 

* Data types: 
    * price should be a float
    * promo price should be a float
* Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 
* Missing values: 
    * Description may be can be infered by the name
    * price. Is there a way we can extract the information from another table?
    * type. Do we need this column for our analysis?
    
This task can be accomoplished by using all the methods you already know.

### Start of the challenge

* Data types:

1.Price should be a float <br>  2.Promo price should be a float

In [669]:
products

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392


In [670]:
products.price.sample(10)

16710          2039
8109           2569
19181    35.998.952
1202            749
8296           2929
8156           2209
18266            30
5827           3769
5855           4069
13284         24.99
Name: price, dtype: object

In [514]:
new_list_price= []
def calculatePrice(value):
    valueSplitted = value.split('.')
    if(len(valueSplitted) > 1):
        if(len(valueSplitted[-1]) > 2):
            newValue = valueSplitted[-1][:-2]
            valueSplitted.pop()
            valueConcatenate = ''.join(valueSplitted)
            value = valueConcatenate+newValue
        else:
            value = value.replace('.','')

    return(value)





      

In [523]:
products = products.assign(n_p = products['price'].str.replace('\.',''))
products = products.assign(n_p_p = products['promo_price'].str.replace('\.',''))
products.sample(10)

<ipython-input-523-03b8e046cd19>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  products = products.assign(n_p = products['price'].str.replace('\.',''))
<ipython-input-523-03b8e046cd19>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  products = products.assign(n_p_p = products['promo_price'].str.replace('\.',''))


,sku,name,desc,price,promo_price,in_stock,type,n_p,n_p_p
5144,PAC1046,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 32GB...",IMac desktop computer 27 inch 5K Retina i5 3.3...,3589,302499,0,"5,74E+15",3589,302499
8251,PAC1580,"Apple iMac 21.5 ""Core i5 3.1GHz 4K Retina | RA...",Desktop computer iMac 21.5-inch 3.1GHz Core i5...,2449,211398,0,1282,2449,211398
18672,WAC0232-A,Open - Wacom pen Digital Pen CTH-470S / 670s S...,Intuos digital pen refitted to Pen and Touch w...,3498,272,0,1229,3498,272
3164,APP1200,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 8GB ...",IMac desktop computer 27 inch 5K Retina i5 33G...,2629,220558,0,"5,74E+15",2629,220558
4032,APP1387,"Apple iMac 27 ""Core i5 3.3GHz Retina 5K | 8GB ...",IMac desktop computer 27 inch 8GB RAM 256GB Re...,2929,278958,0,"5,74E+15",2929,278958
26,APP0280,Apple HDMI to HDMI Cable 1.8m,HMDI to HDMI Cable 1.8m.,25,2400,0,1325,25,2400
18817,DOD0011,Hi-Res Dodocool 24-bit stereo earphones Black,Stereo headphones with remote control and micr...,1999,1299,1,5384,1999,1299
14892,TUC0310,"Tucano Nido Hard-Shell Case MacBook Pro 13 ""(L...",Protective cover with slip rubber feet 13 inch...,299,2499,1,13835403,299,2499
8449,PAC0925,"Apple iMac 27 ""Core i5 3.2GHz Retina 5K | 32GB...",IMac desktop computer 27 inch Retina 5K RAM 32...,2849,229899,0,1282,2849,229899
16273,APP2082,"Apple MacBook Pro 13 ""Core i5 with Touch Bar 3...",New MacBook Pro 13 inch Touch Bar 31 GHz Core ...,200559,188500,0,"1,02E+12",200559,188500


In [558]:
def convertPrice(value):
    valuestring = value.join('\.')
    newdecimals = ('00')
    if(len(valuestring) >2):
        newValuew = valuestring[-1][:-2]
    else:
        value = value.append(newdecimals)
        
    return(value)
            


In [553]:
new_list_p= []

for any in products['n_p']:
    any= convertPrice(str(any))
    new_list_p.append(any)


In [554]:
products['n_p'] = new_list_p

In [715]:
import pandas as pd

products = pd.read_csv("products.csv")
print(products)

           sku                                           name  \
0      RAI0007              Silver Rain Design mStand Support   
1      APP0023              Apple Mac Keyboard Keypad Spanish   
2      APP0025               Mighty Mouse Apple Mouse for Mac   
3      APP0072  Apple Dock to USB Cable iPhone and iPod white   
4      KIN0007    Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM   
...        ...                                            ...   
19321  BEL0376        Belkin Travel Support Apple Watch Black   
19322  THU0060   Enroute Thule 14L Backpack MacBook 13 "Black   
19323  THU0061    Enroute Thule 14L Backpack MacBook 13 "Blue   
19324  THU0062     Enroute Thule 14L Backpack MacBook 13 "Red   
19325  THU0063   Enroute Thule 14L Backpack MacBook 13 "Green   

                                                    desc  price promo_price  \
0           Aluminum support compatible with all MacBook  59.99     499.899   
1              USB ultrathin keyboard Apple Mac Spanish.     

In [716]:
products = products[products['price'].notna()]


In [719]:
products = products[products['promo_price'].notna()]

In [717]:
products['price'] = products.price.apply(lambda x : x +'.00' if x.count('.') == 0 else x)
products['price'] = products.price.apply(lambda x: x  + '0' if x[-2]=='.' else x)
products['price'] = products.price.apply(lambda x: str(float(x.replace('.',''))/1000) if ( (x[-4]=='.') & (x.count('.')==2)) else x)
products['price'] = products.price.apply(lambda x: str(float(x.replace('.',''))/10000) if ( (x[-4]=='.') & (x.count('.')==1)) else x)

In [720]:
products['promo_price'] = products.promo_price.apply(lambda x : x +'.00' if x.count('.') == 0 else x)
products['promo_price'] = products.promo_price.apply(lambda x: x  + '0' if x[-2]=='.' else x)
products['promo_price'] = products.promo_price.apply(lambda x: str(float(x.replace('.',''))/1000) if ( (x[-4]=='.') & (x.count('.')==2)) else x)
products['promo_price'] = products.promo_price.apply(lambda x: str(float(x.replace('.',''))/10000) if ( (x[-4]=='.') & (x.count('.')==1)) else x)

In [722]:
products['price'] = products.price.astype(float)

In [723]:
products['price'] = products.price.round(decimals=2)

In [726]:
products['promo_price'] = products.promo_price.astype(float)

In [727]:
products['promo_price'] = products.promo_price.round(decimals=2)

In [728]:
products['promo_price'] 

0        59.99
1        59.00
2        59.00
3        25.00
4        34.99
         ...  
19321    29.99
19322    69.95
19323    69.95
19324    69.95
19325    69.95
Name: promo_price, Length: 19280, dtype: float64



Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 

In [680]:
print(products)

           sku                                           name  \
0      RAI0007              Silver Rain Design mStand Support   
1      APP0023              Apple Mac Keyboard Keypad Spanish   
2      APP0025               Mighty Mouse Apple Mouse for Mac   
3      APP0072  Apple Dock to USB Cable iPhone and iPod white   
4      KIN0007    Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM   
...        ...                                            ...   
19321  BEL0376        Belkin Travel Support Apple Watch Black   
19322  THU0060   Enroute Thule 14L Backpack MacBook 13 "Black   
19323  THU0061    Enroute Thule 14L Backpack MacBook 13 "Blue   
19324  THU0062     Enroute Thule 14L Backpack MacBook 13 "Red   
19325  THU0063   Enroute Thule 14L Backpack MacBook 13 "Green   

                                                    desc  price promo_price  \
0           Aluminum support compatible with all MacBook  59.99     499.899   
1              USB ultrathin keyboard Apple Mac Spanish.     

In [681]:
products.sku.duplicated().sum()

8747

In [684]:
products.shape

(19326, 7)

In [698]:
products = products.drop_duplicates('sku')


In [688]:
products.shape

(10579, 7)

Dealing with missing values

In [566]:
products.isna().any()

sku            False
name           False
desc            True
price          False
promo_price    False
in_stock       False
type            True
n_p            False
n_p_p          False
dtype: bool

In [567]:
products.desc.isna().sum()

7

In [578]:

products[['name','desc','sku']]

,name,desc,sku
0,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,RAI0007
1,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,APP0023
2,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,APP0025
3,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,APP0072
4,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,KIN0007
...,...,...,...
19321,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,BEL0376
19322,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,THU0060
19323,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,THU0061
19324,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,THU0062


In [659]:
import pandas as pd

products = pd.read_csv("products.csv")
print(products)

           sku                                           name  \
0      RAI0007              Silver Rain Design mStand Support   
1      APP0023              Apple Mac Keyboard Keypad Spanish   
2      APP0025               Mighty Mouse Apple Mouse for Mac   
3      APP0072  Apple Dock to USB Cable iPhone and iPod white   
4      KIN0007    Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM   
...        ...                                            ...   
19321  BEL0376        Belkin Travel Support Apple Watch Black   
19322  THU0060   Enroute Thule 14L Backpack MacBook 13 "Black   
19323  THU0061    Enroute Thule 14L Backpack MacBook 13 "Blue   
19324  THU0062     Enroute Thule 14L Backpack MacBook 13 "Red   
19325  THU0063   Enroute Thule 14L Backpack MacBook 13 "Green   

                                                    desc  price promo_price  \
0           Aluminum support compatible with all MacBook  59.99     499.899   
1              USB ultrathin keyboard Apple Mac Spanish.     

In [689]:
products['new_desc'] = products.desc.fillna('See the product name')
products['desc'] = products['new_desc']
products.drop(['new_desc','type'], axis=1)
#products.desc.isna().sum()

<ipython-input-689-03c3fff4b1d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['new_desc'] = products.desc.fillna('See the product name')
<ipython-input-689-03c3fff4b1d6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['desc'] = products['new_desc']


,sku,name,desc,price,promo_price,in_stock
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1
...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0


In [639]:
import pandas as pd

orderlines = pd.read_csv("orderlines.csv")

Check prices from orderlines to products

In [690]:
(
products
    .assign(check_sku = products['sku'].isin(orderlines['sku']))
)['check_sku'].sum()

7850

In [691]:
products.shape

(10579, 8)

## Brands

The brands csv looks fine, so we can work together with it. 

In [45]:
brands.to_csv(path + 'clean/brands.csv', index=False)